In [11]:

import pandas as pd
import nltk
import datetime
import sys
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import wordpunct_tokenize
from nltk.chunk import ne_chunk
from nltk.corpus import treebank_chunk
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import defaultdict
from operator import itemgetter
from nltk.stem.snowball import SnowballStemmer
from __future__ import division
%matplotlib inline  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
import math
from sklearn.feature_extraction.text import TfidfTransformer

from collections import defaultdict

from gensim import corpora, models, similarities

from __future__ import division

import pandas as pd
import pyLDAvis
import graphlab as gl
import pyLDAvis.graphlab
import pyLDAvis.gensim as gensimvis
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [12]:
##Import and clean data

In [13]:
ls

Applications/             Practice/                 forbes_top.html
Book_NLP_Fletcher.ipynb   Public/                   forbes_top_files/
DAT8/                     Week 7 Practice.ipynb     good.csv
Desktop/                  anaconda/                 green_stream_twitter.py
Documents/                book.csv                  holbrojm.github.io/
Downloads/                cliff_df.csv              movies.html
Dropbox/                  cliff_literature.html     movies_test.html
Fletcher_NLP.ipynb        code_eval.txt             my_project_folder/
FlethcerScrape.ipynb      data/                     myproject/
GitHub/                   df1.csv                   mystuff/
Google Drive/             df_syn1.csv               nltk_data/
Library/                  df_syn12 copy.csv         orange_stream_twitter.py
Logs/                     df_syn12.csv              scikit_learn_data/
MNIST_data/               df_syn2.csv               spark.csv
Metis/                    doc_cluster.pkl           

In [14]:
book_cliff=pd.read_csv('cliff_df.csv',index_col=0)

In [84]:
book_cliff.head()

,title,authors,cn_address,descriptions
0,100 Years of Solitude,Gabriel García Márquez,http://www.cliffsnotes.com/literature/o/100-ye...,"Of all the works by Garc a M rquez , this nove..."
1,12 Years a Slave,Solomon Northup,http://www.cliffsnotes.com/literature/t/twelve...,nan
2,1984,George Orwell,http://www.cliffsnotes.com/literature/n/1984/1...,"In George Orwell's 1984, Winston Smith wrestle..."
3,The 1990s Newbery Medal Winners,NaN,http://www.cliffsnotes.com/literature/n/the-19...,"In 1921, Frederic G. Melcher, coeditor of Publ..."
4,"Absalom, Absalom!",William Faulkner,http://www.cliffsnotes.com/literature/a/absalo...,"In most of Faulkner's earlier fiction, however..."


In [21]:
book_cliff['descriptions']=book_cliff['descriptions'].astype(str)

In [22]:
cliff_title=list(book_cliff['title'])

In [276]:
cliff_syn[3]

'In 1921, Frederic G. Melcher, coeditor of Publishers Weekly and founder of Children\'s Book Week, proposed an award for authors of distinguished American children\'s books to the American Library Association (ALA) meeting of the Children\'s Librarians\' Section. The purpose of the award would be:,Melcher suggested that the award be named after John Newbery, an eighteenth-century English writer, publisher, and bookseller who had a great influence on the development of children\'s literature in Great Britain as well as in the United States. The children\'s librarians accepted Melcher\'s proposal and, in 1922, it was approved by the ALA Executive Board, becoming the first children\'s book award in the world.,First awarded in 1922, the John Newbery Medal is a prestigious award given each year to the author of the book voted the most distinguished contribution to children\'s literature published in the United States during the previous year. The Association for Library Service to Children 

In [23]:
cliff_syn=list(book_cliff['descriptions'])

In [268]:
url='<option value="*">'
urls=[]

In [269]:
for i in cliff_title:
    urls.append(url.replace('*',i))

In [270]:
q=pd.DataFrame()

In [271]:
q['d']=urls

In [272]:
q.to_csv('cliff_titles.csv')

In [24]:
stop_books = stopwords.words('english')
#stop_books+=stopwords.words('spanish')
stop_books += ['.', ',', '(', ')', "'", '"',u'novel',u'books',
               u'published',u'characters',u'works',u'mr.',u'ms.'
              ,u"'s",u"time",u"only",u"n't",u"book",u'life',u'one',
          u'story',u'one',u'story',u'tells',u'things','..',"&","must",'new','like',"she's","he's","gets","-","--","/",":","shes","hes",
              "get","like","--the",'"the','says','two','him.']

In [25]:
texts=[[word for word in document.lower().split() 
        if word not in stop_books] for document in cliff_syn]

In [26]:
frequency=defaultdict(int)

In [27]:
for text in texts:
    for token in text:
        frequency[token] += 1

In [28]:
texts=[[token for token in text if frequency[token]>1] for text in texts]

In [29]:
dictionary = corpora.Dictionary(texts)

In [30]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [31]:
#Models

In [98]:
lsi = models.LsiModel(corpus, id2word=dictionary,num_topics=15) #num_topics=


In [99]:
tfidf=models.TfidfModel(corpus,id2word=dictionary)


In [100]:
lda = models.LdaModel(corpus,id2word=dictionary,passes=5,num_topics=4) #num_topics=

%time

[WARNING] too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


In [101]:
#Search/comparison

In [280]:
cliff_title[124]

'The Great Gatsby'

In [281]:
query=cliff_syn[124]

In [282]:
search = query

In [283]:
search_vec = dictionary.doc2bow(search.lower().split())

In [284]:
#lsi vector search,index
vec_lsi = lsi[search_vec]
index_lsi = similarities.MatrixSimilarity(lsi[corpus])
sims_lsi = index_lsi[vec_lsi]

[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [285]:
#tfidf vector,search and index
vec_tfidf=tfidf[search_vec]
index_tfidf = similarities.MatrixSimilarity(tfidf[corpus])
sims_tfidf=index_tfidf[vec_tfidf]

[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [286]:
#lda model and vector
vec_lda=lda[search_vec]
index_lda=similarities.MatrixSimilarity(lda[corpus])
sims_lda=index_lda[vec_lda]


[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [287]:
#index and search regular model
index = similarities.MatrixSimilarity(corpus)
sims=index[search_vec]

[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [288]:
#get top results

In [289]:
lda_top = sorted(enumerate(sims_lda), key=lambda item: -item[1])
lsi_top = sorted(enumerate(sims_lsi), key=lambda item: -item[1])
tfidf_top = sorted(enumerate(sims_tfidf), key=lambda item: -item[1])
bag_top = sorted(enumerate(sims), key=lambda item: -item[1])

In [300]:
combined=(sims_lsi*.5+sims_lda*.5+sims_tfidf)/2
comb_top= sorted(enumerate(combined), key=lambda item: -item[1])

In [301]:
for i in range(0,10):
    print (cliff_title[comb_top[i][0]],comb_top[i][1],comb_top[i][0])


('The Great Gatsby', 0.99999988, 124)
('Daisy Miller', 0.45887399, 70)
('An American Tragedy', 0.4396151, 21)
("Love's Labour's Lost", 0.4283694, 182)
('The Count of Monte Cristo', 0.42804369, 65)
('Moll Flanders', 0.42648321, 200)
('The Old Man and the Sea', 0.42645082, 217)
('War and Peace', 0.42565638, 313)
('1984', 0.42416412, 2)
('Flowers for Algernon', 0.42406937, 107)


In [302]:
model_top=lsi_top
for i in range(0,10):
    print (cliff_title[model_top[i][0]],model_top[i][1],model_top[i][0])


('The Great Gatsby', 0.99999994, 124)
('Absalom, Absalom!', 0.8727085, 4)
("All the King's Men", 0.86692458, 13)
('The Grapes of Wrath', 0.85961181, 122)
("Faulkner's Short Stories", 0.85567468, 103)
('The Call of the Wild', 0.85045284, 53)
('100 Years of Solitude', 0.8448422, 0)
("O'Connor's Short Stories", 0.84382963, 213)
("The French Lieutenant's Woman", 0.84282929, 111)
('The Devil in the White City', 0.84214622, 77)


In [303]:
df_rec=pd.DataFrame()

In [304]:
df_rec['lda']=sims_lda
df_rec['lsi']=sims_lsi
df_rec['tfidf']=sims_tfidf


In [305]:
#df_rec=df_rec.drop(df_rec.index[204])

In [306]:
df_norm = (df_rec - df_rec.mean()) / (df_rec.max() - df_rec.min())

In [307]:
df_norm['total']=df_norm['lda']+df_norm['lsi']+df_norm['tfidf']

In [308]:
df_norm['title']=cliff_title

In [309]:
df_norm=df_norm.sort_values(by="total",ascending=False)

In [310]:
df_norm.head()

,lda,lsi,tfidf,total,title
124,0.612226,0.316047,0.988500,1.916773,The Great Gatsby
21,0.492150,0.158102,0.008348,0.658600,An American Tragedy
182,0.488780,0.153926,-0.010328,0.632378,Love's Labour's Lost
65,0.488780,0.152995,-0.010504,0.631270,The Count of Monte Cristo
313,0.488780,0.136025,-0.006621,0.618184,War and Peace


In [312]:
lda = models.LdaModel(corpus,id2word=dictionary,passes=10,num_topics=6) #num_topics=


In [313]:
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda, corpus, dictionary)


In [351]:
pyLDAvis.save_html(vis_data,'cliff_lda.html')

In [352]:
pyLDAvis.display(vis_data)


### Clustering

In [171]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [172]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in cliff_syn:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [173]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'

there are 326441 items in vocab_frame


In [174]:
k_stop = stopwords.words('english')
k_stop += ['.', ',', '(', ')', "'", '"',u'novel',u'books',
               u'published',u'characters',u'works',u'mr.',u'ms.'
              ,u"'s",u"time",u"only",u"n't",u"book",u'life',u'one',
          u'story',u'one',u'story',u'tells',u'things','']

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.4,
                                 min_df=0.05, stop_words=k_stop,
                                 use_idf=True, tokenizer=tokenize_and_stem)

%time tfidf_matrix = tfidf_vectorizer.fit_transform(cliff_syn) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 16.4 s, sys: 190 ms, total: 16.6 s
Wall time: 16.5 s
(325, 1278)


In [176]:
terms = tfidf_vectorizer.get_feature_names()

In [177]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)


In [178]:
from sklearn import cluster

In [179]:
dbscan = cluster.DBSCAN(eps=.8)

In [180]:
dbscan.fit(tfidf_matrix)

DBSCAN(algorithm='auto', eps=0.8, leaf_size=30, metric='euclidean',
    min_samples=5, p=None, random_state=None)

In [181]:
clusters=dbscan.labels_.tolist()

In [182]:
films = { 'title': cliff_title,'synopsis': cliff_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [183]:
frame['cluster'].value_counts()

-1    319
 0      6
Name: cluster, dtype: int64

In [184]:
#affinity propagation

In [192]:
affinity_propagation = cluster.AffinityPropagation()

In [193]:
%time affinity_propagation.fit(tfidf_matrix)

CPU times: user 332 ms, sys: 3.65 ms, total: 335 ms
Wall time: 334 ms


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

In [243]:
yo=affinity_propagation.labels_.tolist()

In [251]:
co=affinity_propagation.cluster_centers_indices_


In [254]:
co

array([  1,   7,  14, 100, 119, 175, 193, 199, 234, 303])

In [249]:
#yo

In [255]:
for i in co:
    print cliff_title[i]
    

12 Years a Slave
The Adventures of Tom Sawyer
All the Light We Cannot See
A Farewell to Arms
Go Set a Watchman
Life of Pi
The Metamorphosis and Other Stories
Mockingjay (Book 3 of The Hunger Games Trilogy)
The Plague
Unbroken


In [194]:
clusters=affinity_propagation.labels_.tolist()

In [195]:
films = { 'title': cliff_title,'synopsis': cliff_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [196]:
frame['cluster'].value_counts()

0    76
5    58
2    54
4    52
7    41
9    31
1     6
3     3
8     2
6     2
Name: cluster, dtype: int64

In [327]:
#kmeans

In [315]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 199 ms, sys: 3.82 ms, total: 203 ms
Wall time: 352 ms


In [316]:

clusters = km.labels_.tolist()

In [344]:
films = { 'title': cliff_title,'synopsis': cliff_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [345]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
clusterd=[]
for i in range(num_clusters):    
    yo=[]
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        yo.append(vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'))    
    clusterd.append(yo)

In [346]:
cliff_cluster=pd.DataFrame()

In [347]:
cliff_cluster['cluster']=range(0,5)
cliff_cluster['words']=clusterd

In [348]:
frame=frame.merge(cliff_cluster,on='cluster')

In [349]:
frame.head()

,title,cluster,words
0,100 Years of Solitude,1,"[world, characters, published, writing, wars, ..."
1,The 1990s Newbery Medal Winners,1,"[world, characters, published, writing, wars, ..."
2,"Absalom, Absalom!",1,"[world, characters, published, writing, wars, ..."
3,Alice's Adventures in Wonderland,1,"[world, characters, published, writing, wars, ..."
4,American Poets of the 20th Century,1,"[world, characters, published, writing, wars, ..."


In [350]:
frame.to_csv('cliff_cluster.csv')

In [326]:
frame['cluster'].value_counts()

1    115
2    112
0     47
4     40
3     11
Name: cluster, dtype: int64

In [ ]:
for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')

In [317]:
len(km.cluster_centers_)

5

In [259]:
len(order_centroids[0])

1278

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [323]:
print("Top terms per cluster:")
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print "Cluster %d words:" % i
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print ' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore')
    
    for title in frame.ix[i]['title'].values.tolist():
        print ' %s,' % title

Top terms per cluster:
Cluster 0 words:
 mrs.
 de
 daughter
 lady
 marry
 duke
 The Age of Innocence,
 All's Well That Ends Well,
 The Ambassadors,
 The American,
 As You Like It,
 Bleak House,
 Comedy of Errors,
 The Count of Monte Cristo,
 The Crucible,
 Cyrano de Bergerac,
 A Doll's House,
 Don Quixote,
 Dream of the Red Chamber,
 Emma,
 Hedda Gabler,
 The House of Mirth,
 The Importance of Being Earnest,
 Ivanhoe,
 Le Père Goriot,
 Les Miserables,
 Madame Bovary,
 Major Barbara,
 The Mayor of Casterbridge,
 Measure for  Measure,
 The Merchant of Venice,
 The Merry Wives of Windsor,
 The Misanthrope,
 Mrs. Dalloway,
 Nana,
 Othello,
 Our Town,
 Phaedra,
 The Pickwick Papers,
 The Portrait of a Lady,
 Pride and Prejudice,
 The Red and the Black,
 The Return of the Native,
 Sense and Sensibility,
 St. Joan,
 The Taming of the Shrew,
 Tartuffe,
 The Three Musketeers,
 Two Gentlemen of Verona,
 Vanity Fair,
 Volpone,
 The Way of the World,
 The Wild Duck,
Cluster 1 words:
 world
 charac

In [ ]:
#ward hierarchal clustering

In [203]:
from sklearn.neighbors import kneighbors_graph


In [218]:
connectivity = kneighbors_graph(tfidf_matrix, n_neighbors=5, include_self=False)

In [225]:
ward = cluster.AgglomerativeClustering(n_clusters=6, linkage='ward',connectivity=connectivity)

In [226]:
y=tfidf_matrix.todense()

In [227]:
ward.fit(y)


AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=<325x325 sparse matrix of type '<type 'numpy.float64'>'
	with 1625 stored elements in Compressed Sparse Row format>,
            linkage='ward', memory=Memory(cachedir=None), n_clusters=6,
            n_components=None, pooling_func=<function mean at 0x106642cf8>)

In [228]:
clusters = ward.labels_.tolist()

In [229]:
films = { 'title': cliff_title,'synopsis': cliff_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [230]:
frame['cluster'].value_counts()

0    303
2      6
1      6
4      4
5      3
3      3
Name: cluster, dtype: int64

In [ ]:
#cosine linkage

In [232]:
agglo_cluster = cluster.AgglomerativeClustering(linkage="average", affinity="cosine", n_clusters=4,connectivity=connectivity)

In [233]:
agglo_cluster.fit(y)

AgglomerativeClustering(affinity='cosine', compute_full_tree='auto',
            connectivity=<325x325 sparse matrix of type '<type 'numpy.float64'>'
	with 1625 stored elements in Compressed Sparse Row format>,
            linkage='average', memory=Memory(cachedir=None), n_clusters=4,
            n_components=None, pooling_func=<function mean at 0x106642cf8>)

In [234]:
clusters = agglo_cluster.labels_.tolist()

In [235]:
films = { 'title': cliff_title,'synopsis': cliff_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [236]:
frame['cluster'].value_counts()

0    322
3      1
2      1
1      1
Name: cluster, dtype: int64